In [ ]:
import pymc as pm
import pandas as pd
import arviz as az

import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('Data/apache_sofa_treatment_63.csv', index_col=0)

In [ ]:
df

In [ ]:
sample_poisson = df['APACHE_II']
observed_ndokrw = df[df['Rozpoznania']=='CHF_nieniedokrwienna']['APACHE_II']
observed_dokrw = df[df['Rozpoznania']=='CHF_niedokrwienna']['APACHE_II']
print(sample_poisson.mean().round(2))

In [ ]:
import seaborn as sns

def plot_hist(sample, **options):
    """Plot a histogram of APACHE_II.
    
    sample: sequence of values
    """
    sns.histplot(sample, stat='probability', discrete=True,
                 alpha=0.5, **options)
    
def legend(**options):
    """Make a legend only if there are labels."""
    handles, labels = plt.gca().get_legend_handles_labels()
    if len(labels):
        plt.legend(**options)
        
def decorate_goals(ylabel='Probability'):
    """Decorate the axes."""
    plt.xlabel('APACHE II socre')
    plt.ylabel(ylabel)
    plt.title('Distribution of APACHE II')
    legend()

In [ ]:
plot_hist(sample_poisson)
decorate_goals()

In [ ]:
α = 30
β = 1.8

In [ ]:
sample_gamma = pm.Gamma.dist(α, β)

In [ ]:
def plot_kde(sample, **options):
    """Plot a distribution using KDE.
    
    sample: sequence of values
    """
    sns.kdeplot(sample, cut=0, **options)

def decorate_rate(ylabel='Likelihood'):
    """Decorate the axes."""
    plt.xlabel('Apache per patient')
    plt.ylabel(ylabel)
    plt.title('Distribution of APACHE scoring rate')
    legend()

In [ ]:
plot_kde(pm.draw(sample_gamma, 1000), label='gamma')
decorate_rate()

In [ ]:
with pm.Model() as model1:
    mu = pm.Gamma('mu', α, β)
    idata = pm.sample_prior_predictive(1000)


In [ ]:
sample_prior = idata.prior['mu']
sample_prior.mean()

In [ ]:
len(sample_prior)

In [ ]:
plot_kde(sample_prior[0], color='gray', label='prior')
decorate_rate('Density')

In [ ]:
with pm.Model() as model2:
    μ = pm.Gamma('mu', α, β)
    apache = pm.Poisson('APACHE', μ)
    idata2 = pm.sample_prior_predictive(1000)

In [ ]:
pm.model_to_graphviz(model2)

In [ ]:
idata2

In [ ]:
sample_prior_pred = idata2.prior['APACHE']

In [ ]:
plot_hist(sample_prior_pred[0], label='prior pred')
decorate_goals()

In [ ]:
sample_poisson.mean().round(2), sample_prior_pred.values.mean().round(2)

In [ ]:
sample_poisson.std().round(2), sample_prior_pred.values.std().round(2)

In [ ]:
with pm.Model() as model2:
    μ = pm.Gamma('mu', α, β)
    apache = pm.Poisson('APACHE', μ, observed=[10])
    idata3 = pm.sample(1000)

In [ ]:
len(idata3.posterior['mu'][0])

In [ ]:
sample_posterior = idata3.posterior['mu'][0]
sample_posterior.values.mean().round(2)

In [ ]:
plot_kde(sample_prior[0], label='Prior', color='gray')
plot_kde(sample_posterior.values, label='Posterior')
decorate_rate()

In [ ]:
sample_prior.values.mean().round(2), sample_posterior.values.mean().round(2)

In [ ]:
az.plot_posterior(sample_posterior)
decorate_rate('Density')

In [ ]:
with pm.Model() as model4:
    μ_A = pm.Gamma('mu_A', α, β)
    μ_B = pm.Gamma('mu_B', α, β)
    apache_A = pm.Poisson('apache_A', μ_A, observed=[17])
    apache_B = pm.Poisson('apache_B', μ_B, observed=[10])

In [ ]:
pm.model_to_graphviz(model4)

In [ ]:
with model4:
    idata4 = pm.sample(1000)

In [ ]:
with model4:
    az.plot_trace(idata4)

In [ ]:
mu_A = idata4.posterior['mu_A']
mu_B = idata4.posterior['mu_B']
mu_A.values.mean(), mu_B.values.mean()

In [ ]:
plot_kde(mu_A[0], label='mu_A posterior')
plot_kde(mu_B[0], label='mu_B posterior')
decorate_rate('Density')

In [ ]:
(mu_A.values > mu_B.values).mean()

In [ ]:
observed_ndokrw.values

In [ ]:
with pm.Model() as model5:
    μ_A = pm.Gamma('mu_A', α, β)
    μ_B = pm.Gamma('mu_B', α, β)
    apache_A = pm.Poisson('ndokrw_A', μ_A, observed=observed_ndokrw)
    apache_B = pm.Poisson('dokrw_B', μ_B, observed=observed_dokrw)

In [ ]:
pm.model_to_graphviz(model5)

In [ ]:
with model5:
    idata5 = pm.sample(1000)
    az.plot_trace(idata5)

In [ ]:
mu_A = idata5.posterior['mu_A']
mu_B = idata5.posterior['mu_B']
mu_A.values.mean(), mu_B.values.mean()

In [ ]:
plot_kde(mu_A[0], label='mu_A posterior')
plot_kde(mu_B[0], label='mu_B posterior')
decorate_rate('Density')

What is the probability that patient with ndorw have higher APACHE than patient with docrw?

In [ ]:
(mu_A.values > mu_B.values).mean()

In [ ]:
az.plot_posterior(idata5.posterior['mu_A'][0])
az.plot_posterior(idata5.posterior['mu_B'][0])
decorate_rate('Density')